In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dl-2024/dadosTextosCientificos.tsv


In [2]:
df = pd.read_csv('/kaggle/input/dl-2024/dadosTextosCientificos.tsv',
                   sep='\t',
                   engine='python',
                   encoding='latin-1')

In [3]:
df

,Título_Público,Descricao_pública
0,Açúcar de frutas e aplicação em bolos como sub...,O uso de inovações tecnológicas auxilia as ind...
1,Desenvolver uma linha de farofas com castanhas...,O uso de inovações tecnológicas auxilia as ind...
2,Desenvolvimento de cobertura líquida para sorvete,O Brasil é considerado um país com a maior bio...
3,Desenvolvimento de conserva de tomate cereja,O Brasil é considerado o país com maior biodiv...
4,Desenvolvimento de massa alimentícia enriqueci...,Nos últimos anos a comunidade científica tem d...
...,...,...
2720,Desenvolvimento de ativos biotecnológicos para...,O avanço das tecnologias para estudos genômico...
2721,SIMA MV POWER,Projeto de PD&I para desenvolvimento de um sis...
2722,Talhonameno Ótimo Klabin,"A Klabin S/A é uma empresa brasileira, de cará..."
2723,Tecnologia de Quantificação de Estoque de Carb...,Com a execução do presente projeto espera-se d...


### Usando GPU

In [4]:
import spacy
import torch

# Verificar se a GPU está disponível
if torch.cuda.is_available():
    print("GPU está disponível. Utilizando a GPU.")
    spacy.require_gpu()
else:
    print("GPU não está disponível. Utilizando a CPU.")

# Carregar um modelo do spaCy
nlp = spacy.load("en_core_web_sm") 

# Processar um texto
doc = nlp("Este é um teste para verificar o processamento na GPU.")

# Exibir o resultado
print([token.text for token in doc])

GPU está disponível. Utilizando a GPU.
['Este', 'é', 'um', 'teste', 'para', 'verificar', 'o', 'processamento', 'na', 'GPU', '.']


### pt_core_news_lg

In [5]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 3.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [27]:
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
import string

# Carregar o modelo de português
nlp_spacy = spacy.load("pt_core_news_lg")

### Stanza

In [7]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.7 MB/s eta 0:00:0000:010:01


In [8]:
import stanza
# Baixar o modelo de português do Stanza
stanza.download('pt')

In [9]:
# Carregar o pipeline de português
nlp = stanza.Pipeline('pt')

/opt/conda/lib/python3.10/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, lambda stor

In [10]:
import string
import stanza
import pandas as pd
from spacy.lang.pt.stop_words import STOP_WORDS

## Removendo pontuações

In [11]:
# Função para remover pontuação
def remover_pontuacao(texto):
    if pd.isnull(texto):
        return texto  # Retorna o valor original se for NaN
    return texto.translate(str.maketrans('', '', string.punctuation))

## Removendo Stop words

In [12]:
# Função para remover stopwords
def remover_stopwords(texto):
    if pd.isnull(texto):
        return texto  # Retorna o valor original se for NaN
    doc = nlp(texto)  # Tokeniza o texto com Stanza
    return ' '.join([word.text for sent in doc.sentences for word in sent.words if word.text.lower() not in STOP_WORDS])

## Realizando a lemmatização

In [13]:
# Função para lemmatizar o texto usando Stanza
def lematizar_texto_stanza(texto):
    if pd.isnull(texto):
        return texto  # Retorna o valor original se for NaN
    doc = nlp(texto)
    lemas = [word.lemma for sent in doc.sentences for word in sent.words]
    return ' '.join(lemas)

## Aplicando funções no dataset

In [14]:
# Função que aplica todos os pré-processamentos
def preprocessar_texto(texto):
    # Passo 1: Remover pontuação
    texto_sem_pontuacao = remover_pontuacao(texto)
    
    # Passo 2: Remover stopwords
    texto_sem_stopwords = remover_stopwords(texto_sem_pontuacao)
    
    # Passo 3: Lemmatizar o texto usando Stanza
    texto_lematizado = lematizar_texto_stanza(texto_sem_stopwords)
    
    return texto_lematizado

In [15]:
# Aplicando o pré-processamento às colunas "Título_público" e "Descrição_pública"
df['Titulo_Publico_preprocessado'] = df['Título_Público'].apply(preprocessar_texto)
df['Descricao_publica_preprocessada'] = df['Descricao_pública'].apply(preprocessar_texto)

### Verificando resultado do pré processamento dos textos

In [16]:
print(f"Título original: {df['Título_Público'][0]}")
print(f"Título processado: {df['Titulo_Publico_preprocessado'][0]}")

Título original: Açúcar de frutas e aplicação em bolos como substituto da sacarose
Título processado: Açúcar fruta aplicação bolo substituto sacarose


In [17]:
print(f"Descrição original: {df['Descricao_pública'][0]}")
print(f"Descrição processada: {df['Descricao_publica_preprocessada'][0]}")

Descrição original: O uso de inovações tecnológicas auxilia as indústrias a desenvolverem modelos sustentáveis de produção, produtos e serviços de forma sustentável a partir de recursos de base biológica com papel importante no desenvolvimento econômico do Brasil, dessa maneira, contribui para os Objetivos de Desenvolvimento Sustentável (ODS) estabelecidos pela Organização das Nações Unidas (ONU) especialmente no âmbito do Objetivo 12 (Assegurar padrões de produção e de consumo sustentáveis). Diante desse cenário, a presenta proposta buscará desenvolver e produzir frutose em pó  que substitua a sacarose para atender o mercado de indústrias de alimentos.
Descrição processada: uso inovação tecnológico auxiliar indústria desenvolver modelo sustentável produção produto serviço sustentável recurso base biológico papel importante desenvolvimento econômico Brasil maneira contribuir objetivo Desenvolvimento Sustentável ODS estabelecer Organização Nações Unidas ONU especialmente âmbito objetivo

In [18]:
df.drop(['Título_Público', 'Descricao_pública'], axis=1, inplace=True)

In [19]:
df

,Titulo_Publico_preprocessado,Descricao_publica_preprocessada
0,Açúcar fruta aplicação bolo substituto sacarose,uso inovação tecnológico auxiliar indústria de...
1,desenvolver linha farofo castanha brasileiro a...,uso inovação tecnológico auxiliar indústria de...
2,desenvolvimento cobertura líquido sorvete,Brasil considerar país biodiversidade mundo di...
3,desenvolvimento conservar tomate cerejo,Brasil considerar país biodiversidade mundo di...
4,desenvolvimento massa alimentício enriquecer f...,último ano comunidade científico despertar int...
...,...,...
2720,desenvolvimento ativo biotecnológico aplicação...,avanço tecnologia estudo genômico transcriptôm...
2721,SIMA MV POWER,projeto PDI desenvolvimento inteligente leitur...
2722,talhonameno bom Klabin,Klabin SA empresa brasileiro caráter privado c...
2723,tecnologia Quantificação Estoque Carbono Suzano,execução presente projeto esperase desenvolver...


### Verificando o Dataset

In [20]:
df['Descricao_publica_preprocessada'].isnull().sum()

4

In [21]:
df['Titulo_Publico_preprocessado'].isnull().sum()

3

In [22]:
# Remover entradas com valores nulos nas colunas processadas
df.dropna(subset=['Titulo_Publico_preprocessado', 'Descricao_publica_preprocessada'], inplace=True)

# Verificar se há dados nulos restantes
print(df.isnull().sum())

Titulo_Publico_preprocessado       0
Descricao_publica_preprocessada    0
dtype: int64


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter

## Função para extrair dependências sintáticas e extrair entidades nomeadas (NER)

In [23]:
# Função para extrair as dependências sintáticas
def extract_dependency_counts(doc):
    dep_counts = Counter([token.dep_ for token in doc])
    return dep_counts

# Função para extrair as entidades nomeadas (NER)
def extract_ner_counts(doc):
    ner_counts = Counter([ent.label_ for ent in doc.ents])
    return ner_counts

### Aplicando no Dataset

In [28]:
# Lista para armazenar as features
dep_features = []
ner_features = []

# Aplicar o modelo em cada texto do dataset
for text in df['Descricao_publica_preprocessada']:  
    doc = nlp_spacy(text)
    dep_counts = extract_dependency_counts(doc)
    ner_counts = extract_ner_counts(doc)
    dep_features.append(dep_counts)
    ner_features.append(ner_counts)

### Criar um conjunto único de relações de dependência e labels de NER para vetorização

In [29]:
# Criar um conjunto único de relações de dependência e labels de NER para vetorização
all_dependencies = set(dep for dep_count in dep_features for dep in dep_count.keys())
all_ner_labels = set(ner for ner_count in ner_features for ner in ner_count.keys())

### Convertendo as dependências sintáticas em matriz

In [30]:
# Converter as contagens de dependências em uma matriz
dep_features_matrix = []
for dep_count in dep_features:
    vector = [dep_count.get(dep, 0) for dep in all_dependencies]
    dep_features_matrix.append(vector)

### Convertendo NER em matriz

In [31]:
# Converter as contagens de NER em uma matriz
ner_features_matrix = []
for ner_count in ner_features:
    vector = [ner_count.get(ner, 0) for ner in all_ner_labels]
    ner_features_matrix.append(vector)

In [32]:
# Converter para arrays numpy
dep_features_matrix = np.array(dep_features_matrix)
ner_features_matrix = np.array(ner_features_matrix)

### Standard Scaler

In [33]:
from sklearn.preprocessing import StandardScaler

# Padronizar as features
scaler = StandardScaler()
dep_features_scaled = scaler.fit_transform(dep_features_matrix)
ner_features_scaled = scaler.fit_transform(ner_features_matrix)

# Combinar as features de dependências e NER
combined_features = np.hstack((dep_features_scaled, ner_features_scaled))

# Definir o número de clusters
num_clusters = 3  

# Aplicar o K-means
kmeans_combined = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_combined.fit(combined_features)

# Criar labels para os clusters
clusters_combined = kmeans_combined.labels_

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [34]:
# Adicionar os clusters ao dataframe original
df['Cluster'] = clusters_combined

# Exibir os resultados
for i, text in enumerate(df['Descricao_publica_preprocessada']):  
    print(f"Texto {i+1}: {text}")
    print(f"Cluster atribuído: {clusters_combined[i]}\n")

Texto 1: uso inovação tecnológico auxiliar indústria desenvolver modelo sustentável produção produto serviço sustentável recurso base biológico papel importante desenvolvimento econômico Brasil maneira contribuir objetivo Desenvolvimento Sustentável ODS estabelecer Organização Nações Unidas ONU especialmente âmbito objetivo 12 assegurar padrão produção consumo sustentável cenário presento propor buscar desenvolver produzir frutose pó substituir sacarose atender mercado indústria alimento
Cluster atribuído: 0

Texto 2: uso inovação tecnológico auxiliar indústria desenvolver modelo sustentável produção destacase necessidade pesquisa objetivo desenvolver produto inovador pronto consumo agregação diferenciação sabor especial produto oriundo floresta brasileiro contribuir desenvolvimento bioeconomio Brasil estudo propor avaliar característica produto fim informação nutricional rotulagem acompanhar período vida útil produto empresa busca inovação melhoria processo desenvolver linha produção 

### Enviei uma amostra para o chat gpt para ver os principais tópicos de cada cluster:
#### Cluster 0:
##### Parece que a maioria dos textos nesse cluster está relacionada a biotecnologia, bioeconomia, sustentabilidade, e a produção de alimentos. Textos sobre o Brasil, a biodiversidade, objetivos de desenvolvimento sustentável (ODS), e bioinsumos caem consistentemente nesse cluster.
#### Cluster 1: 
##### Este cluster contém textos que estão mais associados a tecnologia, software, IoT, inteligência artificial (IA), e computação, sugerindo que o K-means identificou uma diferenciação clara entre o setor tecnológico e o setor de sustentabilidade/alimentos.
#### Cluster 2:
##### Este parece ser o menor cluster e contém textos sobre nutrição esportiva, com foco em desempenho físico e mel, o que sugere que os textos nesse cluster têm um tema mais específico, relacionado à saúde, nutrição e esportes.

In [40]:
df

,Titulo_Publico_preprocessado,Descricao_publica_preprocessada,Cluster
0,Açúcar fruta aplicação bolo substituto sacarose,uso inovação tecnológico auxiliar indústria de...,0
1,desenvolver linha farofo castanha brasileiro a...,uso inovação tecnológico auxiliar indústria de...,0
2,desenvolvimento cobertura líquido sorvete,Brasil considerar país biodiversidade mundo di...,0
3,desenvolvimento conservar tomate cerejo,Brasil considerar país biodiversidade mundo di...,0
4,desenvolvimento massa alimentício enriquecer f...,último ano comunidade científico despertar int...,0
...,...,...,...
2720,desenvolvimento ativo biotecnológico aplicação...,avanço tecnologia estudo genômico transcriptôm...,0
2721,SIMA MV POWER,projeto PDI desenvolvimento inteligente leitur...,1
2722,talhonameno bom Klabin,Klabin SA empresa brasileiro caráter privado c...,2
2723,tecnologia Quantificação Estoque Carbono Suzano,execução presente projeto esperase desenvolver...,0


In [41]:
df['Cluster'].value_counts()

Cluster
1    2175
0     485
2      61
Name: count, dtype: int64

# Transfere Learning

In [43]:
!pip install transformers datasets

## BERTimbau Base (aka "bert-base-portuguese-cased")

#### O BERTimbau é um modelo de linguagem pré-treinado baseado na arquitetura BERT (Bidirectional Encoder Representations from Transformers) focado no português brasileiro. Ele foi desenvolvido para abordar diversas tarefas de Processamento de Linguagem Natural (NLP) de maneira eficaz em textos em português, alcançando desempenhos de última geração em diversas tarefas, como:

##### - Reconhecimento de Entidades Nomeadas (Named Entity Recognition - NER): Identificar e classificar nomes de pessoas, organizações, locais, entre outros em textos.
##### - Similaridade Textual de Sentenças (Semantic Textual Similarity - STS): Avaliar a semelhança semântica entre duas frases.
##### - Reconhecimento de Entalhes Textuais (Recognizing Textual Entailment - RTE): Determinar se uma frase implica, contradiz ou é neutra em relação a outra frase.

#### Treinamento e Desempenho
##### O BERTimbau foi treinado em um grande volume de textos em português, incluindo dados do Corpus Brasileiro e da Wikipedia, resultando em um modelo altamente adaptado ao idioma. Assim, ele supera outros modelos multilíngues (como o mBERT e o XLM-R) em tarefas específicas para o português, oferecendo melhorias de desempenho em tarefas de NLP em português brasileiro

#### Documentação:
- https://huggingface.co/neuralmind/bert-base-portuguese-cased

#### Repositório 
- https://github.com/neuralmind-ai/portuguese-bert/

![bert](https://i.imgur.com/JZ7Hynh.png)

### Carrefgando modelo e tokenizer BERTimbau Base Com 12 camadas, 768 dimensões de embeddings e 12 cabeças de atenção.

In [68]:
# Carregar o tokenizer e o modelo do BERTimbau
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

In [69]:
# Verificar se a GPU está disponível e mover o modelo para a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### Preparando os dados

### Função para tokenização dos textos

In [45]:
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

# Função de tokenização
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [46]:
# Preparar dados para o treinamento
texts = df['Descricao_publica_preprocessada'].tolist()
labels = df['Cluster'].tolist() 

# Criar um dataset no formato necessário
dataset = [{"text": text, "label": label} for text, label in zip(texts, labels)]

- O formato necessário é um JSON

In [92]:
dataset[0]

{'text': 'uso inovação tecnológico auxiliar indústria desenvolver modelo sustentável produção produto serviço sustentável recurso base biológico papel importante desenvolvimento econômico Brasil maneira contribuir objetivo Desenvolvimento Sustentável ODS estabelecer Organização Nações Unidas ONU especialmente âmbito objetivo 12 assegurar padrão produção consumo sustentável cenário presento propor buscar desenvolver produzir frutose pó substituir sacarose atender mercado indústria alimento',
 'label': 0}

### Dividindo os dados em 80% para treino e 20% para teste

In [47]:
# Dividir em treino e teste
train_texts, test_texts = train_test_split(dataset, test_size=0.2)

# Tokenizar os textos
train_encodings = tokenizer([item['text'] for item in train_texts], truncation=True, padding=True)
test_encodings = tokenizer([item['text'] for item in test_texts], truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Convertendo para tensores

In [48]:
# Converter para tensores
train_labels = torch.tensor([item['label'] for item in train_texts])
test_labels = torch.tensor([item['label'] for item in test_texts])

### Criando dataset Pytorch

In [49]:
# Criar dataset PyTorch
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, train_labels)
test_dataset = MyDataset(test_encodings, test_labels)

In [51]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Configurando parâmetros de treinamento específicos para realizar um fine-tuning (ajuste fino) do modelo com base no nosso Dataset

In [52]:
# Definir os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir='./results', #Diretório onde os resultados do treinamento (pesos do modelo, checkpoints, etc.) serão salvos.         
    evaluation_strategy="epoch", #Avaliar o modelo ao final de cada época.    
    learning_rate=2e-5,             
    per_device_train_batch_size=16, #número de amostras processadas por vez durante o treinamento
    per_device_eval_batch_size=16, #Batch size usado durante a avaliação. 
    num_train_epochs=3, #Taxa de regularização para prevenir overfitting.           
    weight_decay=0.01,              
)

# Definir o trainer
trainer = Trainer(
    model=model, #O modelo BERT definido anteriormente.                       
    args=training_args, #Parâmetros de treinamento configurados anteriormente                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset            
)

# Treinar o modelo
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.177438
2,No log,0.119833
3,No log,0.114774


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=204, training_loss=0.19150595571480544, metrics={'train_runtime': 280.4178, 'train_samples_per_second': 23.28, 'train_steps_per_second': 0.727, 'total_flos': 1469356881302016.0, 'train_loss': 0.19150595571480544, 'epoch': 3.0})

### Executando o modelo e obtendo os embeddings

In [70]:
# Definir o tamanho do batch
batch_size = 8  # 8 textos são processados de uma vez

# Lista para armazenar os embeddings
all_embeddings = []

# Processar os textos em batches
for i in range(0, len(df['Descricao_publica_preprocessada']), batch_size): #A cada iteração, uma fatia dos textos (batch_texts) é extraída para ser processada.
    batch_texts = df['Descricao_publica_preprocessada'][i:i + batch_size].tolist()
    
    with torch.no_grad(): #Desativar gradientes e definir o modelo em modo de avaliação
        model.eval()

        # Tokenizar os textos no batch atual
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # Mover inputs para GPU/CPU

        # Executar o modelo e obter os embeddings
        outputs = model(**inputs)
        
        # Usar `last_hidden_state` para obter a média dos embeddings dos tokens
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)

        # Armazenar os embeddings
        all_embeddings.append(batch_embeddings.cpu())  # Mover de volta para CPU para liberar GPU

# Concatenar todos os embeddings
embeddings = torch.cat(all_embeddings, dim=0)

#### #Tokenizar os textos no batch atual
- padding=True: Preenche o texto com tokens especiais para garantir que todos tenham o mesmo tamanho.
- truncation=True: Trunca textos longos para o tamanho máximo permitido.
- max_length=512: Define que o tamanho máximo do texto será de 512 tokens.

### Clusterização com 4 clusters baseado em similaridade

In [71]:
# Usar os embeddings para clustering
from sklearn.cluster import KMeans

num_clusters = 4  # Ajuste conforme necessário
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(embeddings)

# Adicionar os clusters ao dataframe original
df['cluster'] = kmeans.labels_

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [74]:
# Amostrar alguns textos de cada cluster
num_samples_per_cluster = 10  # Número de textos por cluster para amostrar

for cluster in df['cluster'].unique():
    print(f"Cluster {cluster}:\n")
    
    # Filtrar os textos pertencentes ao cluster
    texts_in_cluster = df[df['cluster'] == cluster]['Descricao_publica_preprocessada'].head(num_samples_per_cluster)
    
    for i, text in enumerate(texts_in_cluster):
        print(f"Texto {i+1}: {text}")
    print("\n" + "-"*50 + "\n")

Cluster 3:

Texto 1: uso inovação tecnológico auxiliar indústria desenvolver modelo sustentável produção produto serviço sustentável recurso base biológico papel importante desenvolvimento econômico Brasil maneira contribuir objetivo Desenvolvimento Sustentável ODS estabelecer Organização Nações Unidas ONU especialmente âmbito objetivo 12 assegurar padrão produção consumo sustentável cenário presento propor buscar desenvolver produzir frutose pó substituir sacarose atender mercado indústria alimento
Texto 2: uso inovação tecnológico auxiliar indústria desenvolver modelo sustentável produção destacase necessidade pesquisa objetivo desenvolver produto inovador pronto consumo agregação diferenciação sabor especial produto oriundo floresta brasileiro contribuir desenvolvimento bioeconomio Brasil estudo propor avaliar característica produto fim informação nutricional rotulagem acompanhar período vida útil produto empresa busca inovação melhoria processo desenvolver linha produção sustentáve

## Enviei as amostras para o Chat gpt para analisar os tópicos de cada cluster

## Cluster 0: Software e Dispositivos Embarcados
#### Este cluster está focado no desenvolvimento de software embarcado e soluções de tecnologia para diferentes setores, incluindo:

##### - Gerenciamento remoto e segurança de dispositivos.
##### - Integração com recursos de computação em nuvem.
##### - Aplicações de detecção facial e biometria, além de interfaces acessíveis e diagnósticos médicos baseados em dispositivos móveis.
##### - Tecnologias para IoT e computação distribuída, focando em soluções para monitoramento, análise de dados e controle de sistemas.

## Cluster 1: Pesquisa e Inovação Tecnológica
#### Este cluster aborda principalmente projetos de pesquisa e desenvolvimento (P&D) com foco em tecnologia aplicada a diversas áreas, como:

##### - Alimentos: Desenvolvimento de produtos alimentícios com maior valor nutricional, embalagens biodegradáveis e práticas sustentáveis de produção.
##### - Saúde: Pesquisas clínicas sobre novos medicamentos fitoterápicos e biomateriais.
##### - Tecnologias sustentáveis: Desenvolvimento de fertilizantes organominerais, adsorventes e biomateriais com foco na sustentabilidade e redução de resíduos.
##### - Inovação industrial: Utilização de processos de manufatura aditiva e polímeros avançados em setores como eletromobilidade e medicina.

## Cluster 2: Inteligência Artificial e Automação
#### Este cluster concentra-se em tecnologias de ponta, principalmente ligadas à inteligência artificial (IA), automação, e computação em nuvem. Os textos discutem:

##### - Aplicações de IA em energias renováveis, como o uso de sistemas inteligentes para otimizar a produção de energia solar.
##### - Gerenciamento de dados e otimização operacional com a ajuda de gêmeos digitais e ferramentas de computação em borda.
##### - Soluções de automação para gestão e monitoramento de sistemas, com destaque para segurança de dados, monitoramento de dispositivos médicos e redes de telecomunicações.


## Cluster 3: Sustentabilidade e Bioeconomia
#### Os textos deste cluster giram em torno de temas como inovação tecnológica aplicada à sustentabilidade, produção agrícola sustentável e bioeconomia. O foco é o uso de recursos biológicos e processos tecnológicos para o desenvolvimento de produtos e modelos produtivos que respeitem o meio ambiente e promovam o desenvolvimento sustentável. Há uma ênfase em:

##### - Substituição de insumos não sustentáveis por alternativas biológicas (ex.: frutose no lugar de sacarose).
##### - Aproveitamento da biodiversidade brasileira para criar produtos inovadores com impacto econômico e ecológico positivo.
##### - Promoção de cadeias produtivas sustentáveis, incluindo a diversificação de produtos e a valorização de recursos nativos.
##### - Saúde alimentar e funcionalidade, com o desenvolvimento de alimentos nutritivos que atendem às demandas de consumidores conscientes.

# Comparações

In [76]:
df

,Titulo_Publico_preprocessado,Descricao_publica_preprocessada,Cluster,cluster
0,Açúcar fruta aplicação bolo substituto sacarose,uso inovação tecnológico auxiliar indústria de...,0,3
1,desenvolver linha farofo castanha brasileiro a...,uso inovação tecnológico auxiliar indústria de...,0,3
2,desenvolvimento cobertura líquido sorvete,Brasil considerar país biodiversidade mundo di...,0,3
3,desenvolvimento conservar tomate cerejo,Brasil considerar país biodiversidade mundo di...,0,3
4,desenvolvimento massa alimentício enriquecer f...,último ano comunidade científico despertar int...,0,3
...,...,...,...,...
2720,desenvolvimento ativo biotecnológico aplicação...,avanço tecnologia estudo genômico transcriptôm...,0,3
2721,SIMA MV POWER,projeto PDI desenvolvimento inteligente leitur...,1,2
2722,talhonameno bom Klabin,Klabin SA empresa brasileiro caráter privado c...,2,3
2723,tecnologia Quantificação Estoque Carbono Suzano,execução presente projeto esperase desenvolver...,0,3


## Modelo sem o Transfere Learning

In [78]:
df['Cluster'].value_counts()

Cluster
1    2175
0     485
2      61
Name: count, dtype: int64

In [85]:
# 1. Silhouette Score
silhouette_avg = silhouette_score(embeddings, df['Cluster'])
print(f"Silhouette Score: {silhouette_avg}")

# 2. Calinski-Harabasz Index
calinski_harabasz_avg = calinski_harabasz_score(embeddings, df['Cluster'])
print(f"Calinski-Harabasz Index: {calinski_harabasz_avg}")

# 3. Davies-Bouldin Index
davies_bouldin_avg = davies_bouldin_score(embeddings, df['Cluster'])
print(f"Davies-Bouldin Index: {davies_bouldin_avg}")

Silhouette Score: 0.007077100221067667
Calinski-Harabasz Index: 89.02731129136355
Davies-Bouldin Index: 3.4520920868747056


## Modelo com o Transefere Learning

In [80]:
#Clusters com Transfere Learning
df['cluster'].value_counts()

cluster
2    908
3    763
1    706
0    344
Name: count, dtype: int64

In [84]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# 1. Silhouette Score
silhouette_avg = silhouette_score(embeddings, df['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

# 2. Calinski-Harabasz Index
calinski_harabasz_avg = calinski_harabasz_score(embeddings, df['cluster'])
print(f"Calinski-Harabasz Index: {calinski_harabasz_avg}")

# 3. Davies-Bouldin Index
davies_bouldin_avg = davies_bouldin_score(embeddings, df['cluster'])
print(f"Davies-Bouldin Index: {davies_bouldin_avg}")

Silhouette Score: 0.06897551566362381
Calinski-Harabasz Index: 162.4632861260057
Davies-Bouldin Index: 3.2364283442394033


#### 1. Silhouette Score:
Primeira Clusterização **(sem transfer learning)**: 0.007
Segunda Clusterização **(com transfer learning)**: 0.069
Interpretação: O Silhouette Score da segunda clusterização é significativamente melhor. O aumento de 0.007 para 0.069 sugere que a clusterização com transfer learning resultou em uma separação mais clara entre os clusters, onde os pontos de dados estão mais bem agrupados em torno dos seus respectivos centros de cluster.
Conclusão: **A separação dos clusters e a coesão interna são melhores na clusterização com transfer learning.**
#### 2. Calinski-Harabasz Index:
Primeira Clusterização **(sem transfer learning)**: 89.03
Segunda Clusterização **(com transfer learning)**: 162.46
Interpretação: O Calinski-Harabasz Index da segunda clusterização é consideravelmente mais alto, o que indica que os clusters estão mais separados e têm menos sobreposição. **Quanto maior o valor, melhor a separação entre os clusters e menor a variabilidade interna**.
Conclusão: A clusterização com transfer learning tem uma estrutura mais bem definida, com melhor separação entre os clusters.
#### 3. Davies-Bouldin Index:
Primeira Clusterização **(sem transfer learning)**: 3.45
Segunda Clusterização (com transfer learning): 3.24
Interpretação: O Davies-Bouldin Index é menor na clusterização com transfer learning (3.24 vs. 3.45). Quanto menor esse valor, melhor a qualidade da separação entre os clusters, já que ele mede a similaridade entre os clusters — menor valor indica menos sobreposição.
Conclusão: **A clusterização com transfer learning apresentou menos sobreposição entre os clusters, o que indica uma melhor separação.**

In [72]:
# Exibir os textos e seus respectivos clusters
for i, (text, cluster) in enumerate(zip(df['Descricao_publica_preprocessada'], df['cluster'])):
    print(f"Texto {i+1}: {text}")
    print(f"Cluster atribuído: {cluster}\n")

Texto 1: uso inovação tecnológico auxiliar indústria desenvolver modelo sustentável produção produto serviço sustentável recurso base biológico papel importante desenvolvimento econômico Brasil maneira contribuir objetivo Desenvolvimento Sustentável ODS estabelecer Organização Nações Unidas ONU especialmente âmbito objetivo 12 assegurar padrão produção consumo sustentável cenário presento propor buscar desenvolver produzir frutose pó substituir sacarose atender mercado indústria alimento
Cluster atribuído: 3

Texto 2: uso inovação tecnológico auxiliar indústria desenvolver modelo sustentável produção destacase necessidade pesquisa objetivo desenvolver produto inovador pronto consumo agregação diferenciação sabor especial produto oriundo floresta brasileiro contribuir desenvolvimento bioeconomio Brasil estudo propor avaliar característica produto fim informação nutricional rotulagem acompanhar período vida útil produto empresa busca inovação melhoria processo desenvolver linha produção 